In [1]:
class IntegerValue:
    def __init__(self):
        self.values = {}
        
    def __set__(self, instance, value):
        self.values[instance] = int(value)
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            return self.values.get(instance)

In [2]:
import weakref

In [3]:
class IntegerValue:
    def __init__(self):
        self.values = weakref.WeakKeyDictionary()
        
    def __set__(self, instance, value):
        self.values[instance] = int(value)
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            return self.values.get(instance)

In [5]:
class Point:
    x = IntegerValue()


In [6]:
p = Point()

In [7]:
print(hex(id(p)))

0x249ef634048


In [8]:
p.x = 100.1

In [9]:
p.x

100

In [11]:
Point.x.values.keyrefs()

[<weakref at 0x00000249F14582C8; to 'Point' at 0x00000249EF634048>]

In [12]:
del p 

In [13]:
Point.x.values.keyrefs()

[]

In [14]:
class IntegerValue:
    def __init__(self):
        self.values = {}
        
    def __set__(self, instance, value):
        self.values[id(instance)] = int(value)
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            return self.values.get(id(instance))

In [16]:
class Point:
    x = IntegerValue()
    
    def __init__(self, x):
        self.x = x
        
    def __eq__(self, other):
        return isinstance(other, Point) and self.x == other.x

In [17]:
p = Point(10.1)

In [18]:
p.x

10

In [19]:
p.x = 20.2

In [20]:
p.x

20

In [21]:
id(p), Point.x.values

(2516572164168, {2516572164168: 20})

In [22]:
import ctypes

def ref_count(address):
    return ctypes.c_long.from_address(address).value

In [23]:
p_id = id(p)

In [24]:
ref_count(p_id)

1

In [25]:
del p

In [26]:
ref_count(p_id)

495557664

In [27]:
Point.x.values

{2516572164168: 20}

In [28]:
p = Point(10.1)

In [29]:
weak_p = weakref.ref(p)

In [30]:
weak_p

<weakref at 0x00000249F146E818; to 'Point' at 0x00000249F1498C08>

In [31]:
ref_count(id(p))

1

In [33]:
del p

In [34]:
weak_p

<weakref at 0x00000249F146E818; dead>

In [35]:
def obj_destroyed(obj):
    print(f'{obj} is being destroyed')

In [36]:
p = Point(10.1)
w = weakref.ref(p, obj_destroyed)

In [37]:
del p

<weakref at 0x00000249F1474E58; dead> is being destroyed


In [38]:
class IntegerValue:
    def __init__(self):
        self.values = {}
        
    def __set__(self, instance, value):
        self.values[id(instance)] = (weakref.ref(instance, self._remove_object), int(value))
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            return self.values[id(instance)][1]
        
    def _remove_object(self, weak_ref):
        print(f'removing dead entry for {weak_ref}')

In [39]:
class Point:
    x = IntegerValue()

In [40]:
p1 = Point()
p2 = Point()

In [41]:
p1.x, p2.x = 10.1, 100.1

In [42]:
p1.x, p2.x

(10, 100)

In [43]:
ref_count(id(p1)), ref_count(id(p2))

(1, 1)

In [44]:
del p1

removing dead entry for <weakref at 0x00000249F146A908; dead>


In [45]:
class IntegerValue:
    def __init__(self):
        self.values = {}
        
    def __set__(self, instance, value):
        self.values[id(instance)] = (weakref.ref(instance, self._remove_object), int(value))
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            return self.values[id(instance)][1]
        
    def _remove_object(self, weak_ref):
#         reverse_lookup = [key for key, value in self.values.items()
#                          if value[0] is weak_ref]
#         if reverse_lookup:
#             key = reverse_lookup[0]
#             del self.values[key]
            
        for key, value in self.values.items():
            if value[0] is weak_ref:
                del self.values[key]
                break

In [46]:
class Point:
    x = IntegerValue()

In [47]:
p = Point()

In [48]:
p.x = 10.1

In [49]:
p.x

10

In [50]:
Point.x.values

{2516603981192: (<weakref at 0x00000249F14703B8; to 'Point' at 0x00000249F1494D88>,
  10)}

In [51]:
del p

In [52]:
Point.x.values

{}

In [53]:
class Person:
    pass

In [54]:
Person.__dict__

mappingproxy({'__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None})

In [55]:
hasattr(Person.__weakref__, '__get__')

True

In [56]:
hasattr(Person.__weakref__, '__set__')

True

In [57]:
p = Person()

In [58]:
hasattr(p, '__weakref__')

True

In [59]:
print(p.__weakref__)

None


In [60]:
w = weakref.ref(p)

In [61]:
print(p.__weakref__)

<weakref at 0x00000249F147FBD8; to 'Person' at 0x00000249EF633D08>


In [62]:
class Person:
    __slots__ = 'name',

In [63]:
Person.__dict__

mappingproxy({'__module__': '__main__',
              '__slots__': ('name',),
              'name': <member 'name' of 'Person' objects>,
              '__doc__': None})

In [64]:
p = Person()

In [66]:
hasattr(p, '__weakref__')

False

In [67]:
w = weakref.ref(p)

TypeError: cannot create weak reference to 'Person' object

In [68]:
class Person:
    __slots__ = 'name', '__weakref__'

In [69]:
p = Person()

In [70]:
weakref.ref(p)

<weakref at 0x00000249F1488138; to 'Person' at 0x00000249F14EF2C8>

In [71]:
p.__weakref__

<weakref at 0x00000249F1488138; to 'Person' at 0x00000249F14EF2C8>

In [94]:
class ValidString:
    def __init__(self, min_length=0, max_length=255):
        self.data = {}
        self._min_length = min_length
        self._max_length = max_length
        
    def __set__(self, instance, value):
        if not isinstance(value, str):
            raise ValueError ('Value must be a string')
        if len(value) < self._min_length:
            raise ValueError (
                f'Value should be at least {self._min_length} characters.'
            )
        if len(value) < self._min_length:
            raise ValueError (
                f'Value cannot exceed {self._max_length} characters.'
            )     
        self.data[id(instance)] = (weakref.ref(instance, self._finalize_instance), value)
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            value_tuple = self.data.get(id(instance))
            return value_tuple[1]
            
    def _finalize_instance(self, weak_ref):
        reverse_lookup = [key for key, value in self.data.items()
                         if value[0] is weak_ref]
        if reverse_lookup:
            key = reverse_lookup[0]
            del self.data[key]
        

In [95]:
class Person:
    __slots__ = '__weakref__',
    
    first_name = ValidString(1, 100)
    last_name = ValidString(1, 100)
    
    def __eq__(self, other):
        return (
            isinstance(other, Person) and
            self.first_name == other.first_name and
            self.last_name == other.last_name
        )
    
class BankAccount:
    __slots__ = '__weakref__',
    
    account_number = ValidString(5, 255)
    
    def __eq__(self, other):
        return isinstance(other, BankAccount) and self.account_number == other.account_number

In [96]:
p1 = Person()

In [97]:
try:
    p1.first_name = ''
except ValueError as ex:
    print(ex)

Value should be at least 1 characters.


In [98]:
p2 = Person()

In [99]:
p1.first_name, p1.last_name = 'Guido', 'van Rossum'
p2.first_name, p2.last_name = 'Raymond', 'Hettinger'

In [100]:
b1, b2 = BankAccount(), BankAccount()
b1.account_number, b2.account_number = 'Savings', 'Checking'

In [101]:
p1.first_name, p1.last_name

('Guido', 'van Rossum')

In [102]:
p2.first_name, p2.last_name

('Raymond', 'Hettinger')

In [103]:
b1.account_number, b2.account_number

('Savings', 'Checking')

In [104]:
Person.first_name.data

{2516604091512: (<weakref at 0x00000249F1AF9778; to 'Person' at 0x00000249F14AFC78>,
  'Guido'),
 2516604091416: (<weakref at 0x00000249F1904908; to 'Person' at 0x00000249F14AFC18>,
  'Raymond')}

In [105]:
Person.last_name.data

{2516604091512: (<weakref at 0x00000249F1AF92C8; to 'Person' at 0x00000249F14AFC78>,
  'van Rossum'),
 2516604091416: (<weakref at 0x00000249F1904BD8; to 'Person' at 0x00000249F14AFC18>,
  'Hettinger')}

In [106]:
BankAccount.account_number.data

{2516604032008: (<weakref at 0x00000249F1904B38; to 'BankAccount' at 0x00000249F14A1408>,
  'Savings'),
 2516604031192: (<weakref at 0x00000249F1904868; to 'BankAccount' at 0x00000249F14A10D8>,
  'Checking')}

In [107]:
del p1
del p2
del b1
del b2

In [108]:
Person.first_name.data

{}